# **Soccer Coaches Performance Analysis By Evaluated** **Tweets**

In [2]:
!pip install transformers[sentencepiece]
!pip install -U sentence-transformers
!pip install ProfanityDetector
!pip install contractions
!pip install matplotlib
!pip install tweetnlp
!pip install networkx
!pip install pyLDAvis
!pip install snscrape
!pip install keybert
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install spacy
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
Looki

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import snscrape.modules.twitter as sntwitter
from nltk.corpus import stopwords, wordnet
from gensim.utils import simple_preprocess
from nltk.tokenize import TweetTokenizer
from gensim.models import CoherenceModel
from nltk.stem import WordNetLemmatizer
from ProfanityDetector import detector
import matplotlib.colors as mcolors
from nltk.corpus import stopwords
import gensim.corpora as corpora
import torch.nn.functional as F
import matplotlib.pyplot as plt
from itertools import product
import pyLDAvis.gensim_models
from keybert import KeyBERT
from sklearn import metrics
import networkx as nx
import contractions
import pandas as pd
import numpy as np
import wordcloud
import tweetnlp
import pyLDAvis
import random
import string
import gensim
import spacy
import math
import nltk
import math
import sys
import os
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


# Data extraction and twitter crawling

In [5]:
queries = ["eddie howe lang:en since:2022-12-26 until:2023-04-09"]
for query in queries:
    tweets = []
    limit = 50000
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if(len(tweets) > limit):
            break
        else:
            tweets.append(tweet.content)
    # tweets = list(dict.fromkeys(tweetsFile["Tweet"].tolist()))
    df = pd.DataFrame({'Tweet': tweets})
    df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/crawled/{query}.csv", index=False)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Preprocessing the crawled tweets

In [6]:
files = os.listdir('/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/eddie')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
tk = TweetTokenizer()
stopwordsList = stopwords.words('english')
nlp = spacy.load("en_core_web_sm")
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
def finalPreprocess(tweet):
    tweet = re.sub(r'[^\w\s]',' ', tweet)
    tweet = re.sub(r'[^a-zA-Z0-9 ]', ' ', tweet);
    tweet = tweet.lower()
    tweet = [token for token in tk.tokenize(tweet) if not token in stopwordsList]
    tweet = " ".join(tweet)
    preprocessedTweet = []
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(str(tweet)))
    wordnet_tagged = [*map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged)]
    preprocessedTweet = []
    for word, tag in wordnet_tagged:
        if tag is None:
            preprocessedTweet.append(word)
        else:
            preprocessedTweet.append(lemmatizer.lemmatize(word, tag))
    preprocessedTweet = " ".join(preprocessedTweet)
    return preprocessedTweet
def primitivePreprocess(tweet):
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"
    u"\U0001F300-\U0001F5FF"
    u"\U0001F680-\U0001F6FF"
    u"\U0001F1E0-\U0001F1FF"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u'\U00010000-\U0010ffff'
    u"\u200d"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\u3030"
    u"\ufe0f"
    "]+", flags=re.UNICODE)
    tweet = emoji_pattern.sub(r'', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', ' ', tweet)
    tweet = re.sub(r"#([a-zA-Z0-9_]{1,50})", ' ', tweet)
    tweet = re.sub(r"@([a-zA-Z0-9_]{1,50})", ' ', tweet)
    tweet = re.sub(r"&([a-zA-Z0-9_]{1,50})", ' ', tweet)
    return tweet
primitivePreprocessedTweets = []
finalPreprocessedTweets = []
tweetsFile = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/eddie/eddie howe lang:en since:2022-12-26 until:2023-04-09.csv")
tweets = tweetsFile["Tweet"].tolist()
for tweet in tweets:
    tweet = primitivePreprocess(tweet)
    if tweet.lower().find('de zerbi') != -1 and detector(tweet)[1] == False:
        primitivePreprocessedTweets.append(tweet)
# primitivePreprocessedTweets = list(dict.fromkeys(primitivePreprocessedTweets))
for tweet in primitivePreprocessedTweets:
    finalPreprocessedTweets.append(finalPreprocess(tweet))
finalPreprocessedTweets = [(" ".join([token.text_with_ws for token in nlp(tweet) if token.pos_ == "NOUN" or token.pos_ == "VERB" or token.pos_ == "PROPN" or token.pos_ == "ADJ"])) for tweet in finalPreprocessedTweets if isinstance(tweet, str)]
df = pd.DataFrame({"Tweet": primitivePreprocessedTweets, "Preprocessed": finalPreprocessedTweets})
df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/eddie/eddie_preprocessed2.csv", index=False)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date

# Filtering useful tweets out of noisy ones

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
knn = KNeighborsClassifier(n_neighbors=2)
handyLabelledMain = []
handyLabelledPreprocessed = []
handyLabelledFeature = []
handyLabelledNoisy = []
knnNutrimentMain = []
knnNutrimentPreprocessed = []
knnNutrimentFeature = []
tweetsFile = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/labelled/de zerbi.csv")
for index, row in tweetsFile.iterrows():
    if row["Noisy"] == "y" or row["Noisy"] == "n":
        handyLabelledMain.append(row["Tweet"])
        handyLabelledPreprocessed.append(row["Preprocessed"])
        handyLabelledNoisy.append(row["Noisy"])
    else:
        knnNutrimentMain.append(row["Tweet"])
        knnNutrimentPreprocessed.append(row["Preprocessed"])
for tweet in handyLabelledPreprocessed:
    vector = model.encode(tweet)
    handyLabelledFeature.append(vector)
for tweet in knnNutrimentPreprocessed:
    vector = model.encode(tweet)
    knnNutrimentFeature.append(vector)
knnNutrimentFeature = np.array([feature.tolist() for feature in knnNutrimentFeature])
handyNMain = []
handyNPreprocessed = []
handyNFeature = []
handyYMain = []
handyYFeature = []
handyYPreprocessed = []
for i in range(len(handyLabelledNoisy)):
    if handyLabelledNoisy[i] == "n":
        handyNFeature.append(handyLabelledFeature[i])
        handyNMain.append(handyLabelledMain[i])
        handyNPreprocessed.append(handyLabelledPreprocessed[i])
    elif handyLabelledNoisy[i] == "y":
        handyYFeature.append(handyLabelledFeature[i])
        handyYMain.append(handyLabelledMain[i])
        handyYPreprocessed.append(handyLabelledPreprocessed[i])
random.seed(3)
random.shuffle(handyNFeature)
random.shuffle(handyYFeature)
x_train = handyYFeature[:175] + handyNFeature[:250]
x_test = handyYFeature[175:] + handyNFeature[250:]
label_train = 175*["y"] + 250*["n"]
label_test = len(handyYFeature[175:]) * ["y"] + len(handyNFeature[250:])*["n"]
knn.fit(x_train, label_train)
confusion_matrix = metrics.confusion_matrix(label_test, knn.predict(x_test))
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['n', 'y'])
print("KNN Confusion Matrix:")
cm_display.plot()
plt.show()
predictions = knn.predict(knnNutrimentFeature)
knnYPredictsMain = []
knnYPredictsPreprocessed = []
knnNPredictsMain = []
knnNPredictsPreprocessed = []
c = 0
for i in range(len(predictions)):
    if predictions[i] == "y":
        c = c + 1
        knnYPredictsMain.append(knnNutrimentMain[i])
        knnYPredictsPreprocessed.append(knnNutrimentPreprocessed[i])
    else:
        knnNPredictsMain.append(knnNutrimentMain[i])
        knnNPredictsPreprocessed.append(knnNutrimentPreprocessed[i])
print(f"Length of KNN Noisy Predictions: {c}")
length = 0
j = 0
nMainSaver = []
nPreprocessedSaver = []
mainSaver = []
preprocessedSaver = []
noisySaver = []
rfcTestingLabels = []
mixHandyKnnYMain = knnYPredictsMain + handyYMain
mixHandyKnnYPreprocessed = knnYPredictsPreprocessed + handyYPreprocessed
shuffleHandyNMain = handyNMain
shuffleHandyNPreprocessed = handyNPreprocessed
random.seed(3)
temp1 = list(zip(mixHandyKnnYMain, mixHandyKnnYPreprocessed))
temp2 = list(zip(shuffleHandyNMain, shuffleHandyNPreprocessed))
random.shuffle(temp1)
random.shuffle(temp2)
mixHandyKnnYMain, mixHandyKnnYPreprocessed = zip(*temp1)
shuffleHandyNMain, shuffleHandyNPreprocessed = zip(*temp2)
mixHandyKnnYMain, mixHandyKnnYPreprocessed = list(mixHandyKnnYMain), list(mixHandyKnnYPreprocessed)
shuffleHandyNMain, shuffleHandyNPreprocessed = list(shuffleHandyNMain), list(shuffleHandyNPreprocessed)
firstTrainerPreprocessed = shuffleHandyNPreprocessed[:220] + mixHandyKnnYPreprocessed[:450]
firstTrainerMain = shuffleHandyNMain[:220] + mixHandyKnnYMain[:450]
firstNoisyTrainer = 220 * ["n"] + 450 * ["y"]
firstTrainerFeature = []
testerPreprocessed = shuffleHandyNPreprocessed[len(shuffleHandyNPreprocessed) - 170:len(shuffleHandyNPreprocessed)] + mixHandyKnnYPreprocessed[len(mixHandyKnnYPreprocessed) - 200:len(mixHandyKnnYPreprocessed)]
testerMain = shuffleHandyNMain[len(shuffleHandyNMain) - 170:len(shuffleHandyNMain)] + mixHandyKnnYMain[len(mixHandyKnnYMain) - 200:len(mixHandyKnnYMain)]
noisyTester = 170 * ["n"] + 200 * ["y"]
testerFeature = []
noisySolver = []
for tweet in firstTrainerPreprocessed:
    vector = model.encode(tweet)
    firstTrainerFeature.append(vector)
for tweet in testerPreprocessed:
    vector = model.encode(tweet)
    testerFeature.append(vector)
while j<7:
    print(f"RFC Confusion Matrix {j+1}:")
    labelledPreproccesed = []
    labelledNoisy = []
    unlabelledPreproccesed = []
    unlabelledMain = []
    if j==0:
        for i in range(len(firstNoisyTrainer)):
            labelledPreproccesed.append(firstTrainerPreprocessed[i])
            noisySolver.append(firstNoisyTrainer[i]+"***"+firstTrainerPreprocessed[i]+"***"+firstTrainerMain[i])
        for i in range(len(knnNPredictsMain)):
            unlabelledMain.append(knnNPredictsMain[i])
            unlabelledPreproccesed.append(knnNPredictsPreprocessed[i])
    else:
        for counter in range(len(mainSaver[j-1])):
            if noisySaver[j-1][counter] == "y" or noisySaver[j-1][counter] == "n":
                labelledPreproccesed.append(preprocessedSaver[j-1][counter])
                labelledNoisy.append(noisySaver[j-1][counter]+"***"+preprocessedSaver[j-1][counter]+"***"+mainSaver[j-1][counter])
            else:
                unlabelledPreproccesed.append(preprocessedSaver[j-1][counter])
                unlabelledMain.append(mainSaver[j-1][counter])
    labelledFeatures = []
    unlabelledFeatures = []
    for tweet in labelledPreproccesed:
        vector = model.encode(tweet)
        labelledFeatures.append(vector)
    for tweet in unlabelledPreproccesed:
        vector = model.encode(tweet)
        unlabelledFeatures.append(vector)
    labelledFeatures = np.array([feature.tolist() for feature in labelledFeatures])
    unlabelledFeatures = np.array([feature.tolist() for feature in unlabelledFeatures])
    if j==0:
        x_train = firstTrainerFeature
        y_train = noisySolver
        x_test = testerFeature
        y_test = noisyTester
        for item in y_test:
            rfcTestingLabels.append(item)
    else:
        x_train = labelledFeatures
        y_train = labelledNoisy
    rfcTrainPreprocessed = []
    rfcTrainMain = []
    rfcTrainNoisy = []
    for item in y_train:
        rfcTrainNoisy.append(item.split("***")[0])
        rfcTrainPreprocessed.append(item.split("***")[1])
        rfcTrainMain.append(item.split("***")[2])
    rfc = RandomForestClassifier(max_depth=2, random_state=0)
    rfc.fit(x_train, rfcTrainNoisy)
    rfcPredictions = rfc.predict(unlabelledFeatures)
    rfcPredictedMain = []
    rfcPredictedPreprocessed = []
    rfcPredictedNoisy = []
    for i in range(len(rfcPredictions)):
        rfcPredictedMain.append(unlabelledMain[i])
        rfcPredictedPreprocessed.append(unlabelledPreproccesed[i])
        if rfcPredictions[i] == 'n':
            rfcPredictedNoisy.append("n")
            nMainSaver.append(unlabelledMain[i])
            nPreprocessedSaver.append(unlabelledPreproccesed[i])
            length = length + 1
        else:
            rfcPredictedNoisy.append("")
    confusion_matrix = metrics.confusion_matrix(rfcTestingLabels, rfc.predict(testerFeature))
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['n', 'y'])
    cm_display.plot()
    plt.show()
    transferPreprocessed = []
    transferMain = []
    transferNoisy = []
    for k in range(len(rfcTrainNoisy)):
        if rfcTrainNoisy[k] == "n":
            transferPreprocessed.append(rfcTrainPreprocessed[k])
            transferMain.append(rfcTrainMain[k])
            transferNoisy.append(rfcTrainNoisy[k])
    NQuantity  = [120, 117, 110, 85, 65, 45, 30] 
    YQuantity  = [570, 540, 480, 420, 300, 250, 200]
    rfcQuantity = [80, 75, 70, 65, 60, 55, 50]
    finalMain =  [rfcPredictedMain[i] for i in range(len(rfcPredictedMain)) if rfcPredictedNoisy[i] == "n"][:rfcQuantity[j]] + [rfcPredictedMain[i] for i in range(len(rfcPredictedMain)) if rfcPredictedNoisy[i] != "n"] + mixHandyKnnYMain[:YQuantity[j]] + transferMain[:NQuantity[j]]
    finalPreprocessed = [rfcPredictedPreprocessed[i] for i in range(len(rfcPredictedPreprocessed)) if rfcPredictedNoisy[i] == "n"][:rfcQuantity[j]] + [rfcPredictedPreprocessed[i] for i in range(len(rfcPredictedPreprocessed)) if rfcPredictedNoisy[i] != "n"] + mixHandyKnnYPreprocessed[:YQuantity[j]] + transferPreprocessed[:NQuantity[j]]
    finalNoisy =  rfcQuantity[j] * ["n"] + len([rfcPredictedMain[i] for i in range(len(rfcPredictedMain)) if rfcPredictedNoisy[i] != "n"]) * [""] + YQuantity[j] * ["y"] + transferNoisy[:NQuantity[j]]
    mainSaver.append(finalMain)
    preprocessedSaver.append(finalPreprocessed)
    noisySaver.append(finalNoisy)
    j = j + 1
labelledFeatures = []
unlabelledFeatures = []
unlabelledPreproccesed = []
unlabelledMain = []
for i in range(len(noisySaver[6])):
    if noisySaver[6][i] != "n" and noisySaver[6][i] != "y":
        unlabelledMain.append(mainSaver[6][i])
        unlabelledPreproccesed.append(preprocessedSaver[6][i])
trainer = shuffleHandyNPreprocessed[:len(shuffleHandyNPreprocessed)-200] + nPreprocessedSaver[:len(nPreprocessedSaver)-50] + mixHandyKnnYPreprocessed[:len(mixHandyKnnYPreprocessed)-1990] 
for tweet in trainer:
    vector = model.encode(tweet)
    labelledFeatures.append(vector)
for tweet in unlabelledPreproccesed:
    vector = model.encode(tweet)
    unlabelledFeatures.append(vector)
labelledFeatures = np.array([feature.tolist() for feature in labelledFeatures])
rfc.fit(labelledFeatures, len(shuffleHandyNPreprocessed[:len(shuffleHandyNPreprocessed)-200]) * ["n"] + len(nPreprocessedSaver[:len(nPreprocessedSaver)-50]) * ["n"] + len(mixHandyKnnYPreprocessed[:len(mixHandyKnnYPreprocessed)-1990]) * ["y"])
unlabelledFeatures = np.array([feature.tolist() for feature in unlabelledFeatures])
rfcNextPredictions = rfc.predict(unlabelledFeatures)
confusion_matrix = metrics.confusion_matrix(rfcTestingLabels, rfc.predict(testerFeature))
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['n', 'y'])
cm_display.plot()
print("RFC Confusion Matrix (By Being Trained Under All Gained Data):")
plt.show()
length2 = len([prediction for prediction in rfcNextPredictions if prediction == "n"])
print(f"Length of RFC Useful Predictions: {length + length2}")
knnMain = []
knnPreprocessed = []
for counter in range(len(knnYPredictsMain)):
    knnMain.append(knnYPredictsMain[counter])
    knnPreprocessed.append(knnYPredictsPreprocessed[counter])
rfcMain = []
rfcPreprocessed = []
for i in range(len(nPreprocessedSaver)):
    rfcMain.append(nMainSaver[i])
    rfcPreprocessed.append(nPreprocessedSaver[i])
mlpMainNutriment = []
mlpPreprocessedNutriment = []
for i in range(len(rfcNextPredictions)):
    if rfcNextPredictions[i] == "n":
        rfcMain.append(unlabelledMain[i])
        rfcPreprocessed.append(unlabelledPreproccesed[i])
    else:
        mlpMainNutriment.append(unlabelledMain[i])
        mlpPreprocessedNutriment.append(unlabelledPreproccesed[i])
temp1 = list(zip(rfcMain, rfcPreprocessed))
temp2 = list(zip(knnMain, knnPreprocessed))
temp3 = list(zip(handyYMain, handyYPreprocessed))
temp4 = list(zip(handyNMain, handyNPreprocessed))
random.seed(3)
random.shuffle(temp1)
random.shuffle(temp2)
random.shuffle(temp3)
random.shuffle(temp4)
rfcMain, rfcPreprocessed = zip(*temp1)
rfcMain, rfcPreprocessed = list(rfcMain), list(rfcPreprocessed)
knnMain, knnPreprocessed = zip(*temp2)
knnMain, knnPreprocessed = list(knnMain), list(knnPreprocessed)
handyYMain, handyYPreprocessed = zip(*temp3)
handyYMain, handyYPreprocessed = list(handyYMain), list(handyYPreprocessed)
handyNMain, handyNPreprocessed = zip(*temp4)
handyNMain, handyNPreprocessed = list(handyNMain), list(handyNPreprocessed)
x_train = knnPreprocessed + rfcPreprocessed[:350] + handyYPreprocessed[:400] + handyNPreprocessed[:150]
y_train = len(knnPreprocessed) * ["y"] + 350 * ["n"] + 400 * ["y"] + 150 * ["n"]
x_test = handyNPreprocessed[150:] + handyYPreprocessed[400:]
y_test = len(handyNPreprocessed[150:]) * ["n"] + len(handyYPreprocessed[400:]) * ["y"]
labelledFeatures = []
testerFeatures = []
unlabelledFeatures = []
for tweet in x_train:
    vector = model.encode(tweet)
    labelledFeatures.append(vector)
labelledFeatures = np.array([feature.tolist() for feature in labelledFeatures])
for tweet in x_test:
    vector = model.encode(tweet)
    testerFeatures.append(vector)
testerFeatures = np.array([feature.tolist() for feature in testerFeatures])
for tweet in mlpPreprocessedNutriment:
    vector = model.encode(tweet)
    unlabelledFeatures.append(vector)
unlabelledFeatures = np.array([feature.tolist() for feature in unlabelledFeatures])
np.random.seed(3)
mlp = MLPClassifier(hidden_layer_sizes=(160, 80, 80), alpha=0.07)
# mlp = MLPClassifier(hidden_layer_sizes=(95, 50, 50), alpha=0.065)
mlp.fit(labelledFeatures, y_train)
confusion_matrix = metrics.confusion_matrix(y_test, mlp.predict(testerFeatures))
cm_display = metrics.ConfusionMatrixDisplay( confusion_matrix = confusion_matrix, display_labels = ['n', 'y'])
cm_display.plot()
plt.show()
mlpPredictions = mlp.predict(unlabelledFeatures)
mlpMainNPredictions = []
mlpPreprocessedNPredictions = []
mlpMainYPredictions = []
mlpPreprocessedYPredictions = []
for i in range(len(mlpPredictions)):
    if mlpPredictions[i] == "n":
        mlpMainNPredictions.append(mlpMainNutriment[i])
        mlpPreprocessedNPredictions.append(mlpPreprocessedNutriment[i])
    else:
        mlpMainYPredictions.append(mlpMainNutriment[i])
        mlpPreprocessedYPredictions.append(mlpPreprocessedNutriment[i])
print(f"Length of MLP Useful Predictions: {len([prediction for prediction in mlpPredictions if prediction == "n"])})
rfcIndexes = []
for i in range(len(rfcMain)):
    if rfcMain[i] in handyNMain or rfcMain[i] in mlpMainNPredictions:
        rfcIndexes.append(i)
for i in rfcIndexes:
    index = rfcIndexes.index(i)
    del rfcMain[i-index]
    del rfcPreprocessed[i-index]
handyNIndexes = []
for i in range(len(handyNMain)):
    if handyNMain[i] in mlpMainNPredictions:
        handyNIndexes.append(i)
for i in handyNIndexes:
    index = handyNIndexes.index(i)
    del handyNMain[i-index]
    del handyNPreprocessed[i-index]
knnIndexes = []
for i in range(len(knnMain)):
    if knnMain[i] in handyYMain or knnMain[i] in mlpMainYPredictions:
        knnIndexes.append(i)
for i in knnIndexes:
    index = knnIndexes.index(i)
    del knnMain[i-index]
    del knnPreprocessed[i-index]
handyYIndexes = []
for i in range(len(handyYMain)):
    if handyYMain[i] in mlpMainYPredictions:
        handyYIndexes.append(i)
for i in handyYIndexes:
    index = handyYIndexes.index(i)
    del handyYMain[i-index]
    del handyYPreprocessed[i-index]
wholeMain = rfcMain + knnMain + handyYMain + handyNMain + mlpMainNPredictions + mlpMainYPredictions
wholePreprocessed = rfcPreprocessed + knnPreprocessed + handyYPreprocessed + handyNPreprocessed + mlpPreprocessedNPredictions + mlpPreprocessedYPredictions
wholeNoisy = len(rfcMain + handyNMain+ mlpMainNPredictions) * ["n"] + len(knnMain + handyYMain + mlpMainYPredictions) * ["y"]
df = pd.DataFrame({"Tweet": wholeMain, "Preprocessed": wholePreprocessed, "Noisy": wholeNoisy})
df.to_csv("/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/labelled/DeZerbiFinal.csv", index=False)

# Doing sentiment analysis on collected tweets

In [ ]:
files = os.listdir('/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/labelled/')
nlp = spacy.load('en_core_web_sm')
kw_model = KeyBERT()
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
embModel = tweetnlp.load_model('sentence_embedding')
nerModel = tweetnlp.load_model('ner')
for file in files:
    aspect = "De Zerbi"
    tweetsFile = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/labelled/{file}")
    finalSentiments = []
    tweets = []
    preprocessedTweets = []
    for index, row in tweetsFile.iterrows():
        if row["Noisy"] == "n":
            tweets.append(row["Tweet"])
            preprocessedTweets.append(row["Preprocessed"])
            sentence = row["Tweet"]
            entities = nerModel.predict(sentence)
            persons = []
            for entity in entities:
                if entity["type"] == "person":
                    persons.append(entity['entity'])
            if len(persons) != 2:
                persons = []
            inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
            outputs = absa_model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            probs = probs.detach().numpy()[0]
            rowSentiment = []
            if len(persons) > 0:
                if embModel.similarity(aspect, persons[0]) < embModel.similarity(aspect, persons[1]):
                    aspect2 = persons[0]
                else:
                    aspect2 = persons[1]
                inputs2 = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect2} [SEP]", return_tensors="pt")
                outputs2 = absa_model(**inputs2)
                probs2 = F.softmax(outputs2.logits, dim=1)
                probs2 = probs2.detach().numpy()[0]
                if math.floor(probs[0]*100) > math.floor(probs[2]*100) and math.floor(probs2[0]*100) > math.floor(probs2[2]*100):
                    if math.floor(probs2[0]*100) > math.floor(probs[0]*100):
                        sentence = sentence.lower()
                        sentence = re.sub(r'de zerbi', ' jack ', sentence)
                        sentence = re.sub(aspect2.lower(), ' joe ', sentence)
                        keyScores = kw_model.extract_keywords(sentence, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=50)
                        keys = [key for key, score in keyScores]
                        edges = []
                        doc = nlp(sentence)
                        tokens = []
                        for token in doc:
                            tokens.append(token.text)
                            for child in token.children:
                                edges.append(('{0}'.format(token),'{0}'.format(child)))
                        graph = nx.Graph(edges)
                        for key in keys:
                            if key != "jack" and key != "joe":
                                if key in tokens:
                                    if "jack" in graph and "joe" in graph and key in graph:
                                        if nx.has_path(graph, source= "jack", target= key) and nx.has_path(graph, source= "joe", target= key):
                                            path1 = nx.shortest_path_length(graph, source= "jack", target= key)
                                            path2 = nx.shortest_path_length(graph, source= "joe", target= key)
                                            if path1 > path2:
                                                sentence = re.sub(rf"{key}", ' ', sentence)
                        sentence = re.sub(r'jack', 'De Zerbi', sentence)
                        sentence = re.sub(r"joe", aspect2, sentence)
                        inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
                        outputs = absa_model(**inputs)
                        probs = F.softmax(outputs.logits, dim=1)
                        probs = probs.detach().numpy()[0]
                        rowSentiment.append(math.floor(probs[0]*100))
                        rowSentiment.append(math.floor(probs[2]*100))
                    else:
                        rowSentiment.append(math.floor(probs[0]*100))
                        rowSentiment.append(math.floor(probs[2]*100))
                elif math.floor(probs[0]*100) < math.floor(probs[2]*100) and math.floor(probs2[0]*100) < math.floor(probs2[2]*100):
                    if math.floor(probs2[2]*100) > math.floor(probs[2]*100):
                        sentence = sentence.lower()
                        sentence = re.sub(r'de zerbi', ' jack ', sentence)
                        sentence = re.sub(aspect2.lower(), ' joe ', sentence)
                        keyScores = kw_model.extract_keywords(sentence, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=50)
                        keys = [key for key, score in keyScores]
                        edges = []
                        doc = nlp(sentence)
                        tokens = []
                        for token in doc:
                            tokens.append(token.text)
                            for child in token.children:
                                edges.append(('{0}'.format(token),'{0}'.format(child)))
                        graph = nx.Graph(edges)
                        for key in keys:
                            if key != "jack" and key != "joe":
                                if key in tokens:
                                    if "jack" in graph and "joe" in graph and key in graph:
                                        if nx.has_path(graph, source= "jack", target= key) and nx.has_path(graph, source= "joe", target= key):
                                            path1 = nx.shortest_path_length(graph, source= "jack", target= key)
                                            path2 = nx.shortest_path_length(graph, source= "joe", target= key)
                                            if path1 > path2:
                                                sentence = re.sub(rf"{key}", ' ', sentence)
                        sentence = re.sub(r'jack', 'De Zerbi', sentence)
                        sentence = re.sub(r"joe", aspect2, sentence)
                        inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
                        outputs = absa_model(**inputs)
                        probs = F.softmax(outputs.logits, dim=1)
                        probs = probs.detach().numpy()[0]
                        rowSentiment.append(math.floor(probs[0]*100))
                        rowSentiment.append(math.floor(probs[2]*100))
                    else:
                        rowSentiment.append(math.floor(probs[0]*100))
                        rowSentiment.append(math.floor(probs[2]*100))
                else:
                    rowSentiment.append(math.floor(probs[0]*100))
                    rowSentiment.append(math.floor(probs[2]*100))
            else:
                if math.floor(probs[0]*100) != math.floor(probs[2]*100):
                    rowSentiment.append(math.floor(probs[0]*100))
                    rowSentiment.append(math.floor(probs[2]*100))
                else:
                    sentence = re.sub(r'[^\w\s]', ' ', sentence)
                    inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
                    outputs = absa_model(**inputs)
                    probs = F.softmax(outputs.logits, dim=1)
                    probs = probs.detach().numpy()[0]
                    rowSentiment.append(math.floor(probs[0]*100))
                    rowSentiment.append(math.floor(probs[2]*100))
            if rowSentiment[0] > rowSentiment[1]:
                finalSentiments.append("LSS")
            elif rowSentiment[1] > rowSentiment[0]:
                finalSentiments.append("MSS")
            else:
                finalSentiments.append("") 
    counter = 0
    while(counter < len(tweets)):
        if finalSentiments[counter] == "":
            tweets.pop(counter)   
            preprocessedTweets.pop(counter)    
            finalSentiments.pop(counter) 
        counter = counter + 1      
    df = pd.DataFrame({"Tweet": tweets, "Preprocessed": preprocessedTweets, "Sentiment": finalSentiments})
    df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/{file}", index=False)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

# Doing unsupervised topic modelling and running LDA method on filtered tweets


In [ ]:
def deleter(tweet):
    tweet = re.sub(r"/", " / ", tweet)
    tweet = re.sub(r",", " , ", tweet)
    t = ner.ner(tweet)
    propn_tweet = []
    for i in t:
        if i['type'] == 'person' or i['type'] == 'group':
            e = i['entity'].strip()
            if e not in propn_tweet:
                propn_tweet.append(e)
    if len(propn_tweet) < 4:
        return 1
    tokens = word_tokenize(tweet)
    tags = nltk.pos_tag(tokens, tagset = "universal")
    tags = [text for (text, pos) in tags if nlp(text.lower()).has_vector and len(text)>2 and text.isalnum() and text not in punctuation and pos == "NOUN"]
    counter = 0
    for i in range(len(propn_tweet)):
        if i != len(propn_tweet)-1:
            if counter == 3:
                return 0
            else:
                e1 = propn_tweet[i]
                e2 = propn_tweet[i+1]
                regex = f"{re.escape(e1)}(.+?\n*){re.escape(e2)}"
                mch = re.search(regex, tweet)
                if mch:
                    items = mch.group(1).split()
                    flag = 1
                    for item in items:
                        if item in tags and len(propn_tweet[i+1:]) < 4:
                            return 1
                        elif item in tags:
                            flag = 0
                    if flag == 1:
                        counter += 1
                    elif counter != 0:
                        counter = 0
    return 0

tweetsFile = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/ldamss_preprocessed.csv")
preprocessed = tweetsFile.Preprocessed.values.tolist()
tweets = tweetsFile.Tweet.values.tolist()

tweets2 = []
preprocessed2 = []
for i in range(len(tweets)):
    if deleter(tweets[i]) == 1:
        tweets2.append(tweets[i])
        preprocessed2.append(preprocessed[i])
tweets = tweets2
preprocessed = preprocessed2

In [ ]:
names = spacy.load("en_core_web_sm")
propnames = tweetnlp.load_model('ner')

makeSame = [["juventus", "juve"], ["barcelona", "barca"], ["manchester", "manchester city", "city"], ["potter", "graham", "graham potter"], ["boely", "todd", "todd boehly"], 
["bloom", "tony", "tony bloom"], ["trossard", "leandro trossard", "leandro"], ["mitoma", "kaoru", "kaoru mitoma"], ["guardiola", "pep", "pep guardiola"], ["webster", "adam webster"], ["gilmour", "billy", "billy gilmour"], ["ensico", "julio", "julio ensico"], 
["lallana", "adam lallana"], ["estupinan", "perwis", "perwis estupinan"], ["marsch", "solly", "solly marsch"], ["allister", "macallister", "mac allister", "alexis mac allister", "alexis macallister", "alexis"], ["gross", "pascal", "pascal gross"], 
["welbeck", "danny", "danny welbeck"], ["ferguson", "evan ferguson", "evan"], ["lamptey", "tariq", "tariq lamptey"], ["mwepu", "enock", "enock mwepu"], ["caicedo", "moises", "moises caicedo"], ["sarmiento", "jeremy", "jeremy sarmiento"],["tottenham", "spur", "spurs"]]
combined = [item for lis in makeSame for item in lis if item!= lis[0]]


for i in range(len(preprocessed)):
    preprocessed[i] = preprocessed[i].replace("de zerbi", "")
    preprocessed[i] = preprocessed[i].replace("dezerbi", "")
    preprocessed[i] = preprocessed[i].replace("zerbi", "")
    preprocessed[i] = preprocessed[i].replace(" de ", "")
    preprocessed[i] = preprocessed[i].replace("roberto", "")
    preprocessed[i] = preprocessed[i].replace("robert", "")
    preprocessed[i] = preprocessed[i].replace("francesco", "")
    preprocessed[i] = preprocessed[i].replace("rdz", "")
    preprocessed[i] = preprocessed[i].replace("brighton", "")
    preprocessed[i] = preprocessed[i].replace("bha", "")

def makeNutriment(tweet):
    t = propnames.ner(tweet)
    propn_tweet = []
    for i in t:
        if i['type'] == 'person' or i['type'] == 'group':
            e = i['entity'].strip()
            en = e.lower()
            ent = " ".join(en.split())
            propn_tweet.append(ent)
    return propn_tweet
propn_allTweets = map(makeNutriment, preprocessed)
properN = []
for item in propn_allTweets:
    properN.append(item)

nouns = []
for tweet in preprocessed:
  token_pos = []
  for token in names(tweet):
    if token.pos_ == 'NOUN':
      token_pos.append(token.text.lower())
  nouns.append(token_pos)

def make_same(mlist):
    for j in range(len(mlist)):
        for i in range(len(mlist[j])):
            if mlist[j][i] in combined:
                for lis in makeSame:
                    if mlist[j][i] in lis:
                        mlist[j][i] = lis[0]
    return mlist

uniN = make_same(nouns)
uniP = make_same(properN)

propNSaver = []
for i in range(len(preprocessed)):
    for prop in uniP[i]:
        if prop not in uniN[i]:
            uniN[i].append(prop)
            if prop != "" and prop != " " and len(prop) > 2:
                propNSaver.append(prop)
        elif prop != "" and prop != " " and len(prop) > 2:
            propNSaver.append(prop)
    eachModifier = []
    for j in range(len(uniN[i])):
        if uniN[i][j] not in eachModifier and uniN[i][j] != "" and uniN[i][j] != " " and len(uniN[i][j]) > 2:
            eachModifier.append(uniN[i][j])
    uniN[i] = eachModifier
token_tweets = uniN

wholeUniques = [token for item in token_tweets for token in item]
copy = wholeUniques
wholeUniques = []
for item in copy:
    if item not in wholeUniques:
        wholeUniques.append(item)
freq = {}
for item in wholeUniques:
    freq[item] = 0
def makeDictionary(desiredlist):
    for tweet in desiredlist:
        for item in tweet:
            freq[item] += 1
    return dict(sorted(freq.items(), key=lambda x:x[1]))
correctTweets = []
propNSaver = list(dict.fromkeys(propNSaver))
for tweet in preprocessed:
    tweet = re.sub(" +", " ", tweet)
    tweet = tweet.split(" ")
    cTweet = []
    for item in tweet:
        if item in propNSaver:
            cTweet.append(item)
    correctTweets.append(cTweet)
preprocessed = correctTweets
for i in range(len(token_tweets)):
    for item in preprocessed[i]:
        if item not in token_tweets[i]:
            token_tweets[i].append(item)
preprocessed = token_tweets
nouns_propN_Dict = makeDictionary(preprocessed)

nouns_propN_c1 = []
for k,v in nouns_propN_Dict.items():
    if v <= 4:
        nouns_propN_c1.append(k)

nouns_propN_e1 = []
for tweet in preprocessed:
    temp = [i for i in tweet if i not in nouns_propN_c1]
    nouns_propN_e1.append(temp)

prepared_tweets = []
indexes = []
for i in range(len(nouns_propN_e1)):
    if len(nouns_propN_e1[i]) != 0:
        prepared_tweets.append(nouns_propN_e1[i])
        indexes.append(i)

for item in wholeUniques:
    if freq[item] == 1 or freq[item] == 0 or freq[item] == 2 or freq[item] == 3 or freq[item] == 4:
        del freq[item] 
for item,value in freq.items():
    freq[item] = 0
prepared_tweets_Dict = makeDictionary(prepared_tweets)
words = []
freq = []
for k,v in prepared_tweets_Dict.items():
    words.append(k)
    freq.append(v)

def create_dictionary(words):
    return corpora.Dictionary(words)
id2word=create_dictionary(prepared_tweets)

def create_document_matrix(tokens, id2word):
  corpus = []
  for text in tokens:
    corpus.append(id2word.doc2bow(text))
  return corpus
corpus=create_document_matrix(prepared_tweets,id2word)

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            num_topics=25,
                                            random_state=100,
                                            chunksize=200,
                                            alpha=1.5,
                                            passes=100,
                                            per_word_topics=True,
                                            id2word=id2word)

def get_lda_topics(model, num_topics, top_n_words):
    word_dict = {}
    for i in range(num_topics):
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
    return pd.DataFrame(word_dict)
get_lda_topics(lda_model,25,10)

dictionary = id2word
def get_tweet_topic(tweet):
    topic_dist = lda_model.get_document_topics(tweet, minimum_probability=0.0)
    topic = max(topic_dist, key=lambda x: x[1])[0]
    return topic
topics = []
for item in prepared_tweets:
    bow_vector = dictionary.doc2bow(item)
    topics.append(get_tweet_topic(bow_vector))
copy = tweets
tweets = []
for i in range(len(copy)):
    if i in indexes:
        tweets.append(copy[i])
df = pd.DataFrame({'Tweet': tweets, 'Topic': topics})
df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/BSProject/datasets/topics25_pre.csv", index=False)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=15)
vis

# Summarization